<a href="https://colab.research.google.com/github/VisarBuza/data-processing-and-visualization2021-22/blob/main/phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook we preprocess a data set of Amazon Prime Movies and TV shows.

The dataset can be found [here](https://www.kaggle.com/shivamb/amazon-prime-movies-and-tv-shows).

In [623]:
import pandas as pd 
import collections

df = pd.read_csv("/amazon_prime_titles.csv")

## Dataset exploration and modification

### Data types
The data set has 12 columns.

**type** and **rating** are categorical data types.

**release_year** is a numerical data type.

**show_id**, and **rating** are alphanumerical data types

In [624]:
len(df.columns)

12

In [625]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [626]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [627]:
df.nunique()

show_id         9668
type               2
title           9668
director        5774
cast            7927
country           86
date_added        84
release_year     100
rating            24
duration         219
listed_in        518
description     9414
dtype: int64

## Data quality
After looking through the data we can find several issues with it.

First we can see that the date_added, and country colums are provided only for 155 and 672 records respectively. We can go ahead and drop these columns since they provide no additional value.

The director and cast values are missing for some of the records.

In [628]:
df.describe(include = 'all')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
count,9668,9668,9668,7586,8435,672,155,9668.000000,9331,9668,9668,9668
unique,9668,2,9668,5774,7927,86,84,NaN,24,219,518,9414
top,s7010,Movie,Call Time: The Finale,Mark Knight,Maggie Binkley,United States,"March 30, 2021",NaN,13+,1 Season,Drama,1
freq,1,7814,1,113,56,253,14,NaN,2117,1342,986,15
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.341849,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.922482,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1920.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN


We can also check if the data set has duplicate rows.

This dataset has no duplicate rows.

In [629]:
duplicate = df[df.duplicated()]
duplicate.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [630]:
df.loc[df['director'] == '1']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6930,s6931,Movie,Title Post onboarding 8,1,1,NaN,NaN,2021,18+,61 min,Action,Title Post onboarding 8
7127,s7128,Movie,Act 4 - Title before 1C onboarding 2,1,1,NaN,NaN,2021,ALL,61 min,Action,1
7128,s7129,Movie,Act 4 - Title before 1C onboarding 1,1,1,NaN,NaN,2021,ALL,61 min,Action,1
7139,s7140,Movie,Clip: 1,1,1,NaN,NaN,2021,18+,3 min,Action,clip1
7238,s7239,Movie,Clip: ACT 4 - Title after removing 1C - 5,1,1,NaN,NaN,2021,ALL,61 min,Action,ACT 4 - Title after removing 1C - 5
7239,s7240,Movie,Clip: ACT 4 - Title after removing 1C - 2,1,1,NaN,NaN,2021,ALL,61 min,Action,ACT 4 - Title after removing 1C - 2 a
7245,s7246,Movie,ACT 4 - Title after removing 1C - 1,1,1,NaN,NaN,2021,ALL,61 min,Action,ACT 4 - Title after removing 1C -
7258,s7259,Movie,Clip: ACT 4 - Title after removing 1C - 4,1,1,NaN,NaN,2021,7+,61 min,Comedy,ACT 4 - Title after removing 1C - 44
8501,s8502,Movie,Act 6 - Title 1,1,1,NaN,NaN,2021,ALL,61 min,Action,Act 6 - Title 1
8502,s8503,Movie,Act 5 - Title 1,1,1,NaN,NaN,2021,ALL,61 min,Action,Act 5 - Title 1


Some of the values for director have values of 1, so we can go ahead and drop those records since they bring no value.

In [631]:
df = df[df.director != '1']

In [632]:
df.loc[df['rating'] == 'AGES_18_']


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
5132,s5133,Movie,100 Streets,Jim O'Hanlon,"Idris Elba, Gemma Arterton, Charlie Creed-Mile...",NaN,NaN,2017,AGES_18_,94 min,"Action, Drama, Suspense","100 Streets is a powerful ensemble drama, foll..."
5150,s5151,Movie,M.F.A.,Natalia Leite,"Francesca Eastwood, Clifton Collins Jr., Leah ...",NaN,NaN,2017,AGES_18_,92 min,"Drama, Horror, Suspense",Francesca Eastwood gives a breakout performanc...
5734,s5735,Movie,Plug Love,Derek Scott,"Murda Pain, Jessica Ryan, Sino Harris, Lance W...",NaN,NaN,2017,AGES_18_,115 min,"Drama, Romance",Ucult Studios presents a woman realizes that b...


Some of the values for rating can be normalized as follows:

NOT_RATE and NR can be mapped to UNRATED

ALL_AGES can be mapped to ALL

AGES_16_ can be mapped to 16+

AGES_18_ can be mapped to 18+

In [633]:
df['rating'] = df['rating'].fillna('ALL')
df['rating'] = df.apply(lambda x : 'UNRATED' if "NOT_RATE" in x['rating'] else x['rating'], axis = 1)
df['rating'] = df.apply(lambda x : 'UNRATED' if "NR" in x['rating'] else x['rating'], axis = 1)
df['rating'] = df.apply(lambda x : 'ALL' if "ALL_AGES" in x['rating'] else x['rating'], axis = 1)
df['rating'] = df.apply(lambda x : '16+' if "AGES_16_" in x['rating'] else x['rating'], axis = 1)
df['rating'] = df.apply(lambda x : '18+' if "AGES_18_" in x['rating'] else x['rating'], axis = 1)

## Missing values

There are a lot of missing values, especially the country and date_added columns, we can safely remove them.

In [634]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8980
date_added      9497
release_year       0
rating             0
duration           0
listed_in          0
description        0
dtype: int64

In [635]:
df.drop(columns=['date_added', 'country'], inplace = True)

For the rating column, we can estimate that they are applicable to all ages and will mark them as **ALL**

In [636]:
df['rating'] = df['rating'].fillna('ALL')

In [637]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
release_year       0
rating             0
duration           0
listed_in          0
description        0
dtype: int64

In [638]:
df.describe(include="all")

,show_id,type,title,director,cast,release_year,rating,duration,listed_in,description
count,9652,9652,9652,7570,8419,9652.000000,9652,9652,9652,9652
unique,9652,2,9652,5773,7927,NaN,18,219,518,9405
top,s7010,Movie,Call Time: The Finale,Mark Knight,Maggie Binkley,NaN,13+,1 Season,Drama,"Arpo was designed as a commando robot, built a..."
freq,1,7798,1,113,56,NaN,2117,1342,986,15
mean,NaN,NaN,NaN,NaN,NaN,2008.320866,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,18.931135,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1920.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,2007.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,2016.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN


## Dimensionality reduction

We have columns that are not needed for further processing of the data, so we can go ahead and remove them.


In [639]:
df.drop(columns=['description','show_id'], inplace = True)

## Transformation

The duration column has the values as string, and it differs based on type of the record.
If the record is a movie, the column of duration accounts for the minutes of the duration.
If the record is a tv show, the column of duration accounts for the number of seasons that the show has.

**TV Show**: 1 Season

**Movie** : 113 min

We can normalize the duration field for movies by removing the min part of the string.
For tv shows we can add a new feature labeled season_count which is the normalized value of how many seasons that tv show has.

In [640]:
df['season_count'] = df.apply(lambda x : x['duration'].split(" ")[0] if "Season" in x['duration'] else "", axis = 1)
df['duration'] = df.apply(lambda x : x['duration'].split(" ")[0] if "Season" not in x['duration'] else "", axis = 1)

## Feature creation

All of the movies or tv shows presented here fall under some genre in the **listed_in** column. These are delimited with **' , '**. New columns can be created to represent a primary genre, secondary genre and so forth.

In [641]:
df[['genre 1', 'genre 2', 'genre 3', 'genre 4', 'genre 5']] = df['listed_in'].str.split(',', expand=True)
df.drop(columns=['listed_in'], inplace=True)

## Binarization

In the dataset we are using, there are only two columns for type, so we can change the name to IsMovie and binarize the values

In [642]:
df.loc[df['type'] == 'Movie', 'type'] = 1
df.loc[df['type'] == 'TV Show', 'type'] = 0
df.rename(columns={'type': 'is_movie'}, inplace = True)
df.head()

,is_movie,title,director,cast,release_year,rating,duration,season_count,genre 1,genre 2,genre 3,genre 4,genre 5
0,1,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,ALL,113,,Comedy,Drama,None,None,None
1,1,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,13+,110,,Drama,International,None,None,None
2,1,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,ALL,74,,Action,Drama,Suspense,None,None
3,1,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,ALL,69,,Documentary,None,None,None,None
4,1,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,ALL,45,,Drama,Fantasy,None,None,None


Here is a sample of the preprocessed data set

In [643]:
df.sample(n = 30)

,is_movie,title,director,cast,release_year,rating,duration,season_count,genre 1,genre 2,genre 3,genre 4,genre 5
3171,1,Bad Ass,Craig Moss,"Danny Trejo, Charles S. Dutton, Patrick Fabian...",2012,R,90,,Action,None,None,None,None
8024,1,Nasha (Kannada),Amit Saxena,"Poonam Pandey, Shivam Patil, Sheetal Singh, Vi...",2021,18+,114,,Romance,Suspense,None,None,None
4315,1,Old MacDonald Had a Farm & More Kids Songs - S...,Brett Jubinville,NaN,2019,ALL,42,,Animation,Kids,None,None,None
8430,1,RiffTrax: Astro-Zombies,Michael J. Nelson,"Michael J. Nelson, Kevin Murphy, Bill Corbett,...",2016,13+,92,,Comedy,Horror,None,None,None
589,0,The Kicks,NaN,"Sixx Orange, Isabella Acres, E'myri Crutchfiel...",2016,ALL,,1,Kids,None,None,None,None
7895,1,Cruel Intentions 2,Roger Kumble,"Amy Adams, Mimi Rogers",2000,R,87,,Drama,None,None,None,None
779,1,"The 5th King- Iran ""The Blade"" Barkley Story","Luis Libran, Carl June","Iran Bakley, Yvonne Barkley, Mark Breland, Lui...",2019,13+,66,,Documentary,Drama,Sports,None,None
2172,1,How We Got Gay,Marc de Guerre,NaN,2016,13+,43,,Documentary,LGBTQ,None,None,None
488,1,The Perfect Match (1987),Mark Deimel,NaN,1988,13+,89,,Comedy,None,None,None,None
4211,1,15-Minute Full Body Blast 3.0 Workout,NaN,Maggie Binkley,2017,ALL,18,,Fitness,None,None,None,None


## Insights
In the next section we provided some insights about how the dataset might be useful

##TV shows

In [644]:
tvshows = df[df['is_movie'] == 0]
tvshows.drop(columns=['director'], inplace = True)
tvshows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1854 entries, 17 to 9666
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_movie      1854 non-null   object
 1   title         1854 non-null   object
 2   cast          1385 non-null   object
 3   release_year  1854 non-null   int64 
 4   rating        1854 non-null   object
 5   duration      1854 non-null   object
 6   season_count  1854 non-null   object
 7   genre 1       1854 non-null   object
 8   genre 2       1060 non-null   object
 9   genre 3       376 non-null    object
 10  genre 4       27 non-null     object
 11  genre 5       12 non-null     object
dtypes: int64(1), object(11)
memory usage: 188.3+ KB


##Movies

In [645]:
movies = df[df['is_movie'] == 1]
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7798 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_movie      7798 non-null   object
 1   title         7798 non-null   object
 2   director      7570 non-null   object
 3   cast          7034 non-null   object
 4   release_year  7798 non-null   int64 
 5   rating        7798 non-null   object
 6   duration      7798 non-null   object
 7   season_count  7798 non-null   object
 8   genre 1       7798 non-null   object
 9   genre 2       4769 non-null   object
 10  genre 3       1680 non-null   object
 11  genre 4       406 non-null    object
 12  genre 5       311 non-null    object
dtypes: int64(1), object(12)
memory usage: 852.9+ KB
